In [12]:
# buffer loop (1st read and write buffer, plus subsequent read and append buffer)

import os
import time
from datetime import datetime
import pandas as pd
from sodapy import Socrata

apptoken = os.environ.get("<apptokenhere>")
domain = "data.melbourne.vic.gov.au"
client = Socrata(domain, apptoken)

interval, hours = 15, 1*24*7*6
runs = int(hours*(60/interval))
ds_id, ds_no = 'vh2v-4nfs', "003"
ds_fname = os.path.join('datasets', ds_no+"_"+ds_id+"__bufferred"+".csv")
uq_fname = os.path.join('datasets', ds_no+"_"+ds_id+"__uniqueBays"+".csv")
ds_col1 = ['st_marker_id', 'status']
ds_col2 = ['bay_id', 'st_marker_id', 'lat', 'lon']
log_fname = os.path.join('datasets', ds_no+"_"+ds_id+"__log"+".txt")
for i in range(runs):
    r1 = client.get_all(ds_id)  # get snapshot of dataset using sodapy api
    df_temp = pd.DataFrame.from_dict(r1)  # read dict to dataframe
    df_temp1 = df_temp[ds_col1]
    df_temp2 = df_temp[ds_col2]
    df_temp1['db_read_time'] = datetime.now()  # add timestampe column
    if i == 0:
        df_temp1.to_csv(ds_fname, mode='w', header=True, index=False)  # first write
        df_temp2.to_csv(uq_fname, mode='w', header=True, index=False)
        o = open(log_fname,'w')
        print(f"{datetime.now()}, dataset retrieved and written to csv", file=o)
        print(f"{datetime.now()}, unique parking bays csv written", file=o)
        o.close()
    else:
        df_temp1.to_csv(ds_fname, mode='a', header=False, index=False)  # subsequent append to csv
        df_temp2prev = pd.read_csv(uq_fname)
        df_temp2new = pd.concat([df_temp2prev, df_temp2], axis='index').drop_duplicates(subset=['st_marker_id'])
        df_temp2new.to_csv(uq_fname, mode='w', header=True, index=False)
        o = open(log_fname,'a')
        print(f"{datetime.now()}, dataset retrieved and appended to csv", file=o)
        print(f"{datetime.now()}, unique parking bays csv updated", file=o)
        o.close()
    time.sleep(60*interval)  # sleep 15min before next read and append

<ipython-input-12-be1d06d613cf>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp1['db_read_time'] = datetime.now()  # add timestampe column
